# Development Scratch

This notebook contains ideas/scratch/notes for development

In [8]:
from availabilities.models import *
from events.models import *
import datetime
import pandas as pd
from collections import defaultdict

In [2]:
FEB_3_9AM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_3_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_4_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_4_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_5_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_5_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [3]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_3_9AM,
    "date_time_end": FEB_3_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_4_2PM,
    "date_time_end": FEB_4_5PM,
    "is_available": False
}

andrew = User.objects.get(username="Andrew")
jessie = User.objects.get(username="Jessie")

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)

# Generates test data: Event times for 2022/2/3 2-5pm, 2022/2/4 2-5pm, 2022/2/5 2-5pm
event_defaults = {
    "members": []
}
event, __ = Event.objects.get_or_create(
    name="Hockey Party",
    host_member=andrew
)
event.members.add(jessie)
event.save()

proposed_times = [
    (FEB_3_2PM, FEB_3_5PM),
    (FEB_4_2PM, FEB_4_5PM),
    (FEB_5_2PM, FEB_5_5PM),
]
for proposed_start, proposed_end in proposed_times:
    EventTime.objects.get_or_create(
        event=event,
        date_time_start=proposed_start,
        date_time_end=proposed_end,
    )

# Peeks
event_time_df = pd.DataFrame(EventTime.objects.filter(event=event).values())
event_time_df

,id,event_id,date_time_start,date_time_end
0,1,1,2022-02-03 14:00:00+00:00,2022-02-03 17:00:00+00:00
1,2,1,2022-02-04 14:00:00+00:00,2022-02-04 17:00:00+00:00
2,3,1,2022-02-05 14:00:00+00:00,2022-02-05 17:00:00+00:00


In [16]:
def get_all_event_participants(event):
    """
    Returns all event members + host_member
    """
    host_member = [event.host_member]
    members = list(event.members.all())
    participants = host_member + members
    return participants


def get_availabilities_for_participants(participants):
    """
    Returns a list of availabilities for all participants
    """
    availabilities = list()
    availability_models = Availability.__subclasses__()
    for availability_model in availability_models:
        availabilities += list(availability_model.objects.filter(user__in=participants))
        
    return availabilities


def check_availabilities(event):
    """
    Checks availability of all members + host against proposed times
    """
    # Gets all members + host member
    participants = get_all_event_participants(event)
    n_particpants = len(participants)
    
    # Gets all availabilities
    availabilities = get_availabilities_for_participants(participants)
    
    # Gets suggested event times
    suggested_event_times = list(EventTime.objects.filter(event=event))
    
    # Compares availabilities against suggested times
    availability_dict = defaultdict(list)
    times_that_work = list()
    for event_time in suggested_event_times:
        for availability in availabilities:
            is_available = availability.check_availability(event_time)
            if is_available:
                availability_dict[event_time.id].append(availability.user.username)  
        everyone_is_available = len(availability_dict[event_time.id]) == n_particpants
        if everyone_is_available:
            times_that_work.append(event_time.id)
                
    return availability_dict, times_that_work
    
check_availabilities(event)


(defaultdict(list, {1: ['Jessie'], 2: ['Andrew'], 3: ['Andrew', 'Jessie']}),
 [3])

In [5]:
availabilities[0].check_availability(event_times[0])

False

In [6]:
availabilities[0].date_time_start

datetime.datetime(2022, 2, 3, 9, 0, tzinfo=<UTC>)

In [7]:
for availability in availabilities:
    for event_time in event_times:
        print(f"Availability: {availability.is_available}")
        print(f"User: {availability.user.username}")
        print(f"Availability Start: {availability.date_time_start}")
        print(f"Availability End: {availability.date_time_end}")
        print(f"Event Start: {event_time.date_time_start}")
        print(f"Event End: {event_time.date_time_end}")
        print(f"Can Work?: {availability.check_availability(event_time)}")
        print("\n")
        

Availability: False
User: Andrew
Availability Start: 2022-02-03 09:00:00+00:00
Availability End: 2022-02-03 17:00:00+00:00
Event Start: 2022-02-03 14:00:00+00:00
Event End: 2022-02-03 17:00:00+00:00
Can Work?: False


Availability: False
User: Andrew
Availability Start: 2022-02-03 09:00:00+00:00
Availability End: 2022-02-03 17:00:00+00:00
Event Start: 2022-02-04 14:00:00+00:00
Event End: 2022-02-04 17:00:00+00:00
Can Work?: True


Availability: False
User: Andrew
Availability Start: 2022-02-03 09:00:00+00:00
Availability End: 2022-02-03 17:00:00+00:00
Event Start: 2022-02-05 14:00:00+00:00
Event End: 2022-02-05 17:00:00+00:00
Can Work?: True


Availability: False
User: Jessie
Availability Start: 2022-02-04 14:00:00+00:00
Availability End: 2022-02-04 17:00:00+00:00
Event Start: 2022-02-03 14:00:00+00:00
Event End: 2022-02-03 17:00:00+00:00
Can Work?: True


Availability: False
User: Jessie
Availability Start: 2022-02-04 14:00:00+00:00
Availability End: 2022-02-04 17:00:00+00:00
Event St